In [1]:
import sys
from pathlib import Path
from datetime import datetime, timedelta
import json
from collections import defaultdict
import pandas as pd
import xxhash
from datasets import load_dataset
from tqdm import tqdm

workspace_root = Path.cwd()
sys.path.insert(0, str(workspace_root / 'src'))

from thesis_pipeline.io.config import load_all_configs

print(f"CC-NEWS download started: {datetime.now().isoformat()}")
print(f"Workspace: {workspace_root}")

CC-NEWS download started: 2025-12-18T22:33:09.306572
Workspace: /Users/stahlma/Desktop/01_Studium/11_Thesis/Data_Experiment


## 1. Load Configuration and Setup

In [2]:
configs = load_all_configs(workspace_root / 'configs')
news_cfg = configs['news']
global_cfg = configs['global']

# Output directory
silver_news_dir = workspace_root / 'data/01_silver/news'
silver_news_dir.mkdir(parents=True, exist_ok=True)

# Date range
start_date = datetime.strptime(global_cfg['validation_run']['period_start'], '%Y-%m-%d')
end_date = datetime.strptime(global_cfg['validation_run']['period_end'], '%Y-%m-%d')
total_days = (end_date - start_date).days + 1

# Sampling config
daily_limit = news_cfg['sampling']['daily_limit_n']
seed = global_cfg['validation_run']['seed_string']
hash_function = news_cfg['sampling']['hash_function']

print(f"Dataset: {news_cfg['source']['hf_dataset']}")
print(f"Date range: {start_date.date()} to {end_date.date()} ({total_days} days)")
print(f"Daily limit: {daily_limit:,} articles")
print(f"Seed: {seed}")
print(f"Hash function: {hash_function}")
print(f"Output: {silver_news_dir.relative_to(workspace_root)}")

Dataset: stanford-oval/ccnews
Date range: 2016-09-01 to 2016-10-31 (61 days)
Daily limit: 10,000 articles
Seed: thesis_sep_oct_2016_v1
Hash function: xxhash64
Output: data/01_silver/news


## 2. Define Hash-Based Sampling Function

In [3]:
def compute_hash(seed: str, url: str, date: str) -> int:
    """Compute deterministic hash from seed, url, and date."""
    combined = f"{seed}||{url}||{date}"
    return xxhash.xxh64(combined.encode('utf-8')).intdigest()

def parse_article_date(article):
    """Extract date from article, return YYYY-MM-DD string or None."""
    # CC-NEWS has 'published_date' field (note: not 'date')
    date_str = article.get('published_date', '')
    if not date_str:
        return None
    
    # Extract just the date part (first 10 chars)
    try:
        return date_str[:10] if len(date_str) >= 10 else None
    except:
        return None

# Test the hash function
test_hash = compute_hash(seed, "https://example.com/article", "2016-09-01")
print(f"\nTest hash: {test_hash}")
print("Hash function ready.")


Test hash: 12131673047118061681
Hash function ready.


## 3. Load HuggingFace Dataset (Streaming Mode)

**Note:** This will stream the dataset without downloading everything. We'll filter for our date range.

In [4]:
print("Loading CC-NEWS dataset...")
print("This will download the 2016 subset (~1-5 GB), then process locally.\n")

# Load dataset in download mode - more reliable than streaming
dataset = load_dataset(
    news_cfg['source']['hf_dataset'],
    name="2016",  # Only download 2016 articles
    split='train'
    # No streaming=True - will download to cache
)

print("✓ Dataset loaded")
print("\nDataset info:")
print(f"  Name: {news_cfg['source']['hf_dataset']}")
print(f"  Year subset: 2016")
print(f"  Total articles: {len(dataset):,}")
print(f"  Mode: Downloaded (cached locally)")

Loading CC-NEWS dataset...
Using DOWNLOAD mode for reliability (streaming has connection issues)
This will download the 2016 subset (~1-2 GB), then process locally.



Resolving data files:   0%|          | 0/479 [00:00<?, ?it/s]

2016_0000.parquet:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

2016_0001.parquet:   0%|          | 0.00/1.92G [00:00<?, ?B/s]

2016_0002.parquet:   0%|          | 0.00/573M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

✓ Dataset loaded

Dataset info:
  Name: stanford-oval/ccnews
  Year subset: 2016
  Total articles: 2,315,442
  Mode: Downloaded (cached locally)


## 4. Inspect Sample Articles

In [5]:
# Peek at first few articles to understand structure
print("Sample articles from dataset:\n")

for i in range(min(3, len(dataset))):
    article = dataset[i]
    print(f"Article {i+1}:")
    print(f"  Date: {article.get('published_date', 'N/A')}")
    print(f"  URL: {article.get('requested_url', 'N/A')[:80]}...")
    print(f"  Title: {article.get('title', 'N/A')[:80]}...")
    print(f"  Text length: {len(article.get('plain_text', ''))} chars")
    print(f"  Language: {article.get('language', 'N/A')}")
    print(f"  Fields: {list(article.keys())}")
    print()

print("=" * 80)

Sample articles from dataset:

Article 1:
  Date: 2016-10-18
  URL: http://www.coventrytelegraph.net/news/its-not-just-aa-plastic-12041279...
  Title: It's not just AA plastic £5 notes that are valuable - have you checked yours? - ...
  Text length: 1524 chars
  Language: en
  Fields: ['requested_url', 'plain_text', 'published_date', 'title', 'tags', 'categories', 'author', 'sitename', 'image_url', 'language', 'language_score', 'responded_url', 'publisher', 'warc_path', 'crawl_date']

Article 2:
  Date: 2016-10-18
  URL: http://www.coventrytelegraph.net/news/coventry-news/one-good-guys-tribute-rugby-...
  Title: 'One of the good guys' - tribute to rugby-loving former police inspector who cap...
  Text length: 2488 chars
  Language: en
  Fields: ['requested_url', 'plain_text', 'published_date', 'title', 'tags', 'categories', 'author', 'sitename', 'image_url', 'language', 'language_score', 'responded_url', 'publisher', 'warc_path', 'crawl_date']

Article 3:
  Date: 2016-10-18
  URL: http

## 5. Stream, Filter, and Sample Articles

**Process:**
1. Stream through dataset
2. Filter articles in our date range (2016-09-01 to 2016-10-31)
3. Compute hash for each article
4. Group by date, keep top-10k smallest hashes per day

In [6]:
# Generate target dates
target_dates = set()
current = start_date
while current <= end_date:
    target_dates.add(current.strftime('%Y-%m-%d'))
    current += timedelta(days=1)

print(f"Target dates: {len(target_dates)} days from {min(target_dates)} to {max(target_dates)}")
print(f"\nProcessing {len(dataset):,} articles from downloaded dataset...")
print(f"This should take 5-10 minutes.\n")

# Data structure: {date: [(hash, article), ...]}
daily_articles = defaultdict(list)

articles_seen = 0
articles_in_range = 0
articles_english = 0

# Process downloaded dataset - much faster than streaming
for article in tqdm(dataset, desc="Processing articles", unit=" articles"):
    articles_seen += 1
    
    # Parse date
    article_date = parse_article_date(article)
    
    # Skip if not in our date range
    if article_date not in target_dates:
        continue
    
    articles_in_range += 1
    
    # Filter for English only
    language = article.get('language', '').lower()
    if language != 'en':
        continue
    
    articles_english += 1
    
    # Get required fields
    url = article.get('requested_url', '')
    if not url:
        continue
    
    # Compute hash
    article_hash = compute_hash(seed, url, article_date)
    
    # Store with hash
    daily_articles[article_date].append((article_hash, article))
    
    # Progress update every 100k articles
    if articles_seen % 100000 == 0:
        dates_covered = len(daily_articles)
        tqdm.write(f"  Seen: {articles_seen:,} | In range: {articles_in_range:,} | English: {articles_english:,} | Dates: {dates_covered}/{len(target_dates)}")

print("\n" + "=" * 80)
print(f"Total articles seen: {articles_seen:,}")
print(f"Articles in date range: {articles_in_range:,}")
print(f"English articles: {articles_english:,}")
print(f"Dates with articles: {len(daily_articles)}/{len(target_dates)}")
print("=" * 80)

Target dates: 61 days from 2016-09-01 to 2016-10-31

Processing 2,315,442 articles from downloaded dataset...
This should take 5-10 minutes.



Processing articles:   4%|▍         | 102615/2315442 [00:05<02:00, 18418.10 articles/s]

  Seen: 100,000 | In range: 98,697 | English: 41,937 | Dates: 49/61


Processing articles: 100%|██████████| 2315442/2315442 [02:00<00:00, 19183.04 articles/s]


Total articles seen: 2,315,442
Articles in date range: 237,603
English articles: 93,737
Dates with articles: 61/61


## 6. Select Top-K Articles per Day

In [7]:
print("\nSelecting top-k articles per day (smallest hashes)...\n")

daily_stats = []

for date in sorted(target_dates):
    if date not in daily_articles:
        print(f"⚠ Warning: No articles found for {date}")
        daily_stats.append({
            'date': date,
            'available': 0,
            'sampled': 0
        })
        continue
    
    articles = daily_articles[date]
    
    # Sort by hash (ascending) and take top-k
    articles.sort(key=lambda x: x[0])
    selected = articles[:daily_limit]
    
    # Update storage with selected articles only
    daily_articles[date] = selected
    
    daily_stats.append({
        'date': date,
        'available': len(articles),
        'sampled': len(selected)
    })

# Display stats
df_stats = pd.DataFrame(daily_stats)
print("Daily sampling summary:")
print(df_stats.to_string(index=False))

print("\n" + "=" * 80)
print(f"Total articles sampled: {df_stats['sampled'].sum():,}")
print(f"Average per day: {df_stats['sampled'].mean():.0f}")
print(f"Days with full quota ({daily_limit:,}): {(df_stats['sampled'] == daily_limit).sum()}/{len(target_dates)}")
print("=" * 80)


Selecting top-k articles per day (smallest hashes)...

Daily sampling summary:
      date  available  sampled
2016-09-01        124      124
2016-09-02        104      104
2016-09-03         53       53
2016-09-04         59       59
2016-09-05         58       58
2016-09-06        106      106
2016-09-07         99       99
2016-09-08        141      141
2016-09-09        111      111
2016-09-10         50       50
2016-09-11        354      354
2016-09-12        288      288
2016-09-13        161      161
2016-09-14        108      108
2016-09-15        151      151
2016-09-16        140      140
2016-09-17         67       67
2016-09-18         72       72
2016-09-19        125      125
2016-09-20        147      147
2016-09-21        175      175
2016-09-22        134      134
2016-09-23        118      118
2016-09-24         50       50
2016-09-25         66       66
2016-09-26        124      124
2016-09-27        141      141
2016-09-28        148      148
2016-09-29        132

## 7. Write Daily Parquet Files

In [8]:
print("\nWriting daily Parquet files...\n")

files_written = []

for date in tqdm(sorted(target_dates), desc="Writing files"):
    if date not in daily_articles or not daily_articles[date]:
        continue
    
    # Extract articles (discard hashes)
    articles = [article for _, article in daily_articles[date]]
    
    # Convert to DataFrame
    df = pd.DataFrame(articles)
    
    # Add date column (ensure it's just YYYY-MM-DD)
    df['date'] = date
    
    # Reorder columns: date first
    cols = ['date'] + [col for col in df.columns if col != 'date']
    df = df[cols]
    
    # Write to Parquet
    output_file = silver_news_dir / f"{date}.parquet"
    df.to_parquet(output_file, compression='snappy', index=False)
    files_written.append(output_file)

print("\n" + "=" * 80)
print(f"✓ Files written: {len(files_written)}")
print(f"✓ Output directory: {silver_news_dir.relative_to(workspace_root)}")
print("=" * 80)


Writing daily Parquet files...



Writing files: 100%|██████████| 61/61 [00:01<00:00, 43.20it/s] 


✓ Files written: 61
✓ Output directory: data/01_silver/news


## 8. Verify Sample File

In [10]:
if files_written:
    # Check a sample file
    sample_file = files_written[len(files_written)//2]
    
    print(f"Sample file: {sample_file.name}")
    print("=" * 80)
    
    df_sample = pd.read_parquet(sample_file)
    
    print(f"Shape: {df_sample.shape}")
    print(f"Columns: {df_sample.columns.tolist()}")
    print(f"\nFirst 3 rows:")
    print(df_sample[['date', 'title', 'requested_url']].head(3).to_string(index=False))
    
    print("\n" + "=" * 80)
else:
    print("⚠ No files written")

Sample file: 2016-10-01.parquet
Shape: (71, 16)
Columns: ['date', 'requested_url', 'plain_text', 'published_date', 'title', 'tags', 'categories', 'author', 'sitename', 'image_url', 'language', 'language_score', 'responded_url', 'publisher', 'warc_path', 'crawl_date']

First 3 rows:
      date                                                                                                                                                     title                                                                                                                 requested_url
2016-10-01                                                                                                                     10 Propositions for Research-Creation                                                           http://quod.lib.umich.edu/j/jep/3336451.0019.206?view=text;rgn=main
2016-10-01                                                                                                                 Judge blocks

## 9. Save Processing Metadata

In [11]:
# Save metadata
metadata = {
    'timestamp': datetime.now().isoformat(),
    'source': {
        'dataset': news_cfg['source']['hf_dataset'],
        'access_method': 'download',  # Changed from streaming
        'year_subset': '2016'
    },
    'period': {
        'start': start_date.strftime('%Y-%m-%d'),
        'end': end_date.strftime('%Y-%m-%d'),
        'days': total_days
    },
    'filters': {
        'language': 'en'
    },
    'sampling': {
        'method': news_cfg['sampling']['method'],
        'hash_function': hash_function,
        'seed': seed,
        'daily_limit': daily_limit
    },
    'processing': {
        'articles_processed': int(articles_seen),
        'articles_in_date_range': int(articles_in_range),
        'articles_english': int(articles_english),
        'total_sampled': int(df_stats['sampled'].sum())
    },
    'output': {
        'directory': str(silver_news_dir.relative_to(workspace_root)),
        'files_written': len(files_written),
        'dates_covered': sorted([f.stem for f in files_written]),
        'missing_dates': sorted(list(target_dates - set([f.stem for f in files_written])))
    },
    'daily_statistics': df_stats.to_dict('records')
}

metadata_file = silver_news_dir / 'processing_metadata.json'
with open(metadata_file, 'w') as f:
    json.dump(metadata, f, indent=2)

print(f"✓ Metadata saved: {metadata_file.relative_to(workspace_root)}")

✓ Metadata saved: data/01_silver/news/processing_metadata.json
